In [1]:
from Q_learing import Q_learning
from fight_env import fight_env
from fight_train_func import rewardFunction, getQTableData
import random
from fightAlgorithm import testQtableData
import pygame


pygame 2.5.2 (SDL 2.28.3, Python 3.9.16)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import math
class QLearningTraining:
    def __init__(self, agent, env,
                 initial_lr=1.0, min_lr=0.01, gamma=0.99):
        self.agent = agent
        self.env = env
        self.initial_lr = initial_lr
        self.min_lr = min_lr
        self.gamma = gamma
        self.rewards_per_episode = []

        

        self.reward_history = []

    def train(self, episodes, useTeacher=False, lrDecrease=500, epsDecrease=500, minLr=0.01, minEps=0.01):
        Lambda_lr = lambda i: max(minLr, min(1, 1.0 - math.log10((i+1)/lrDecrease)))
        Lambda_epsilon = lambda i: max(minEps, min(0.99, 1.0 - math.log10((i+1)/epsDecrease)))
        for episode in range(episodes):
            lr = Lambda_lr(episode) * self.initial_lr
            epsilon = Lambda_epsilon(episode)


            data = self.env.reset()
            
            agentRewards = [0, 0, 0, 0, 0, 0, 0]

            while (True):      
                #self.env.render()          
                updateData = {}
                oldStates = {}
                actions = {}
                end_game = not self.env.not_done()
                
                if (end_game):
                    if (episode % 10 == 0):
                        print(f"time step:{episode + 1}", end=" ")
                        for j in range(1, 7):
                            
                            print(f"player{j} reward: {agentRewards[j]}", end=",")
                        print()
                    break

                for j in range(1, 7):
                    playerID = str(j) + "P"
                    
                        
                    
                    state, _, _ = data[playerID]
                    oldStates[playerID] = state

                    action = self.agent.step(getQTableData(state), epsilon)
                    actions[playerID] = action

                    updateData[playerID] = [action]

                data = self.env.update(updateData)

                for j in range(1, 7):
                    if (self.env.lives[j] == 0):
                        continue
                    
                    playerID = str(j) + "P"
                    old_state = oldStates[playerID]
                    new_state, liveLoss, scoreUp = data[playerID]
                    action = actions[playerID]
                    reward = rewardFunction(old_state, action, scoreUp, liveLoss)
                    
                    self.agent.update(getQTableData(old_state), getQTableData(new_state),\
                                       action, reward, lr, self.gamma)
                    
                    

                    agentRewards[j] += reward
    def show(self):
        
        data = self.env.reset()
        while (True):    
            self.env.render()                
            updateData = {}
            
            for j in range(1, 7):
                playerID = str(j) + "P"

                state, _, _ = data[playerID]
                action = self.agent.step(getQTableData(state))    
                updateData[playerID] = [action]
            data = self.env.update(updateData)  


            if (not self.env.not_done()):
                break


In [3]:
env = fight_env()
obsercationSize = (8, 8, 8, 2, 2, 2)
actionSpace = env.actionSpace
agent = Q_learning(obsercationSize, actionSpace)

print(obsercationSize, actionSpace)

ql = QLearningTraining(agent, env)

(8, 8, 8, 3, 2) ['AIM_RIGHT', 'AIM_LEFT', 'SHOOT', 'FORWARD', 'BACKWARD', 'TURN_RIGHT', 'TURN_LEFT', 'NONE']


In [4]:
ql.train(10000)


time step:1 player1 reward: -7.159999999999892,player2 reward: -7.519999999999884,player3 reward: -7.3599999999998875,player4 reward: -7.459999999999885,player5 reward: -7.179999999999891,player6 reward: -7.099999999999893,


In [5]:


ql.show()
pygame.quit()
ql.agent.save("tank3.pickle")

In [4]:
tank0 = Q_learning(obsercationSize, actionSpace)
tank0.load("tank2.pickle")
showEnv = fight_env(trainMode="plain.tmx")
show = QLearningTraining(tank0, showEnv)

In [6]:
for i in range(10):
    show.show()
pygame.quit()